**CS 6320.002: Natural Language Processing**

**Fall 2020**

**Homework 5 --- 55 points**

**Issued 26 Oct. 2020**

**Due 11:59pm CST 09 Nov. 2020**

**Deliverables:** This completed notebook and your saved model parameters `hw5.encoder` and `hw5.decoder`.

# Getting Started

We are going to implement a sequence-to-sequence model, specifically an encoder-decoder with attention, using recurrent neural networks.

We will be using PyTorch again. In the last assignment, we implemented the neural network from scratch to familiarize ourselves with the bones of how they work. This time, we will be using some of the prebuilt layers in the `torch.nn` module for convenience.

We are using the [E2E Challenge](http://www.macs.hw.ac.uk/InteractionLab/E2E/) dataset for generating restuarant descriptions from *meaning representations*.

Meaning Representation:
```
name[The Eagle],
eatType[coffee shop],
food[French],
priceRange[moderate],
customerRating[3/5],
area[riverside],
kidsFriendly[yes],
near[Burger King]
```

Generated Description:
```
The three star coffee shop, The Eagle, gives families a mid-priced dining experience featuring a variety of wines and cheeses. Find The Eagle near Burger King.
```

In a traditional, non-neural approach, there would be two steps in this task. First, the meaning representations would be converted into a *sentence plan* that would group and reorder them. In the example above, `customerRating`, `name`, `kidsFriendly`, `priceRange`, and `food` are grouped together, in that order, while `near` is in a separate group and `area` is omitted.

The sentence plan would then be *realized* into text. The E2E challenge takes advantage of neural networks to perform this task in only one step, so-called "end-to-end" training. Thus the input to our neural network is a sequence of meaning representations, and the output is a sequence of words, without any intermediate sentence plan.

# Loading and Preprocessing the Data --- 5 points

We will use the very convenient `torchtext` module to load and process our data. 

The code snippet below first prompts you to upload the provided files `train.txt` and `test.txt` to Colab. Take a look at the files. They are formatted as tab-separated values. The first column, labeled "mr", contains a comma-separated list of meaning representations. The second column, labeled "ref" (short for "reference," which is a term used in place of "gold standard" in text generation tasks), contains the sentence(s) that make up the restaurant description.

Fill in the rest of this code snippet following the instructions below.

In [18]:
from google.colab import files
#files.upload()

import torchtext
import nltk
nltk.download('punkt')

START = '<s>'
END = '</s>'
MAX_LEN = 50

src = torchtext.data.Field(tokenize = lambda s : s.split(','),include_lengths=True)
tgt = torchtext.data.Field(
    include_lengths=True, init_token=START, eos_token=END, tokenize = lambda s : nltk.word_tokenize(s),
     lower = True, unk_token = '<unk>', batch_first = True
    )

train_data, test_data = torchtext.data.TabularDataset.splits(
    path='/content/', train='train.txt', test='test.txt', format='tsv',
    fields=[('src', src), ('tgt', tgt)],
    filter_pred=lambda x: len(x.tgt) <= MAX_LEN,
    skip_header=True
    )

src.build_vocab(train_data, max_size=50000)
tgt.build_vocab(train_data, max_size=50000)

print('Source and target vocabulary size')
print(len(src.vocab.stoi))
print(len(tgt.vocab.stoi))

print('\nSource and target vocabulary items')
print(list(src.vocab.stoi.keys())[:10])
print(list(tgt.vocab.stoi.keys())[:10])

print('\nNumber of training pairs: %d' % len(train_data.examples))
print('\nNumber of test pairs: %d' % len(test_data.examples))

print('\nTraining source and target')
print(train_data.examples[0].src)
print(train_data.examples[0].tgt)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Source and target vocabulary size
81
2758

Source and target vocabulary items
['<unk>', '<pad>', ' familyFriendly[yes]', ' area[riverside]', ' eatType[coffee shop]', ' familyFriendly[no]', ' area[city centre]', ' eatType[pub]', ' food[Japanese]', ' food[Italian]']
['<unk>', '<pad>', '<s>', '</s>', '.', 'the', 'is', 'a', ',', 'food']

Number of training pairs: 42021

Number of test pairs: 4660

Training source and target
['name[The Vaults]', ' eatType[pub]', ' priceRange[more than £30]', ' customer rating[5 out of 5]', ' near[Café Adriatic]']
['the', 'vaults', 'pub', 'near', 'café', 'adriatic', 'has', 'a', '5', 'star', 'rating', '.', 'prices', 'start', 'at', '£30', '.']


The code snippet instantiates two `torchtext.data.Field` objects. The first is `src`, representing the input or "source"; the second is `tgt`, representing the output or "target." 

A `Field` object represents some text data that we want to be able to vectorize into `torch.Tensor`'s for a neural network. It can automatically handle the preprocessing steps, such as lowercasing and rare word replacement, which we implemented manually in the previous assignment. However, we do need to set the correct options in the `Field` constructor so that it knows what preprocessing we want to do. 

Some of the options in both the `src` and `tgt` constructors are already filled in. For the source/input, the only thing left to do is tokenization: each meaning representation unit is a token. For example, the string "name[The Eagle]" should be considered a single token.

For the target/output, we also want to do tokenization, but we don't want to simply split on commas --- we want to do proper word tokenization using `nltk.word_tokenize()`. We also want to lowercase all the words and append our trusty start and end meta-tokens `'<s>'` and `'</s>'`.

The code for instantiating the `torchtext.data.TabularDataset` is already completed; it will automatically convert provided files `train.txt` and `test.txt` into dataset objects. Your task is to fill in the constructors for `src` and `tgt` to perform the tokenization and preprocessing steps described above. See [the documentation here](https://torchtext.readthedocs.io/en/latest/data.html#field) for the list of options. 

(HINT: [Lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) are very convenient here; there is an example of one in the `TabularDataset` constructor that filters out any pairs where the target sequence is too long.)

# Building the Neural Network --- 25 points

Now that we have loaded and formatted our data, it's time to build the sequence-to-sequence network. We will be using the `torch.nn` and `torch.nn.functional` modules, which implement most of the common neural network layers and activation functions that we might want to use. Instead of implementing the equations ourselves, as in the last assignment, we can simply use, for example, `torch.nn.Linear` or `torch.nn.NLLLoss`.

For simplicity, we will again be using stochastic, rather than batch or mini-batch gradient descent. Batching introduces some complications for implementation, including increasing the number of dimensions in every tensor and making decoding cumbersome to code up. So our batch size throughout this assignment is simply 1.

## The Encoder

This code snippet implements the encoder part of our encoder-decoder model. Recall from class that an encoder takes the input sequence and computes a sequence of hidden states. The last hidden state is used to initialize the decoder's hidden state; if we were implementing attention (which we aren't in this assignment), we would calculate it using the hidden state sequence. Fill in the encoder code snippet following the instructions below.

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

class Encoder(nn.Module):

  def __init__(self, vocab_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(vocab_size, hidden_size)
    self.GRU = nn.GRU(hidden_size, hidden_size)

  def initialize_hidden_state(self):
    return torch.zeros(1, 1, self.hidden_size, device= 'cuda')

  def forward(self, input_sequence, hidden_state):
    embedded = self.embedding(input_sequence)
    output = embedded
    output, hidden = self.GRU(embedded, hidden_state)
    return (hidden, output)

HIDDEN_SIZE = 128
encoder = Encoder(len(src.vocab), HIDDEN_SIZE).to('cuda')
print(encoder)

Encoder(
  (embedding): Embedding(81, 128)
  (GRU): GRU(128, 128)
)


### The Constructor

First we need to fill in the constructor. We can see that our `Encoder` is a subclass of `nn.Module`, which is PyTorch's base class for neural networks.

This constructor needs to instantiate two things: the embedding layer and the recurrent layer. We are using a GRU as our recurrent unit; it gives a good tradeoff between expressiveness and computational cost.

Fill in the constructor by instantiating an `nn.Embedding` and an `nn.GRU`. The constructors for both of these classes take two arguments: the input dimension size and the output dimension size. See the documentation [here](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding) and [here](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU).

Also notice that there is already a line in the constructor that saves `hidden_size`; we will need it in the next step.

### The Hidden State

Recall that a GRU calculates its hidden state like so:

$\begin{aligned}
\mathbf{r}_t &= \sigma(\mathbf{W}_r \mathbf{x}_t + \mathbf{U}_r \mathbf{h}_{t-1}) \\
\mathbf{z}_t &= \sigma(\mathbf{W}_z \mathbf{x}_t + \mathbf{U}_z \mathbf{h}_{t-1}) \\
\mathbf{n}_t &= \text{tanh}(\mathbf{W}_n \mathbf{x}_t + \mathbf{r}_t \odot \mathbf{U}_n \mathbf{h}_{t-1}) \\
\mathbf{h}_t &= (1 - \mathbf{z}_t) \odot \mathbf{n}_t + \mathbf{z}_t \odot \mathbf{h}_{t-1}
\end{aligned}$

All of these calculations depend on the input $x_t$ and the previous hidden state $h_{t-1}$. But wait... what happens when $t=1$? What is $h_0$?

We need to initialize the hidden state. Note that the hidden state is *not* a parameter, so we don't need to use Xavier initialization like in the previous assignment; the hidden state is one of the "outputs" of our encoder, as we'll see when we implement the forward pass. (In case you're wondering, parameter initialization for our embedding and GRU layers was done automatically by PyTorch when you instantiated them.) It is common to simply initialize the hidden state to be all zeros. This means that at the first time step, the new hidden state $h_1$ is calculated using only the input $x_1$.

Fill in the method `initialize_hidden_state()` to return a hidden state vector of the appropriate shape initialized to all zeros. Two details to keep in mind:
- PyTorch expects an RNN hidden state to have shape `(num_layers * num_directions, batch_size, hidden_size)`; for simplicity, we are using a single direction, single layer GRU and a batch size of just 1 training example. 
- Make sure to set the option `device='cuda'` to make sure computations are done on the GPU, rather than on the CPU, which would take forever. 


### The Forward Pass

The last part of the encoder is the forward pass. As in the previous assignment, the `forward()` method should perform all of the computations needed to transform the input of our encoder into the appropriate output. Unlike in the previous assignment, though, we are using prebuilt PyTorch layers, so we don't need to implement all the equations from scratch; we can simply call `layer_name(input)` to run an input through a layer of the network.

Because an encoder is an RNN-based network, the input is a sequence of words, rather than a single word like in the previous assignment. Also, a word is represented as an index in the vocabulary, rather than one-hot vector; this is basically the sparse version of a one-hot vector. Happily, we don't need to do anything special to account for these differences because PyTorch's embedding and GRU layers can handle sequential inputs and vocabulary indices automatically.

Fill in the method `forward()` to first get the embeddings of the input sequence and then run that sequence of embeddings through the GRU. There are a few wrinkles here:

- The input is a `torch.Tensor` of shape `(sequence_length, 1)`; the output should be the `torch.Tensor` corresponding to the final encoder hidden state after running the input through the GRU.

- The initial hidden state $h_0$ is passed as an argument to the GRU, and the updated hidden state $h_t$ is returned by it. The GRU also returns an `output`, which is the sequence of hidden states $h_1, \ldots, h_t$. If we were implementing attention, we would use this sequence, but since we're not, we can simply ignore it.

## The Decoder

Now let's implement the decoder part of our encoder-decoder model in the code snippet below. Notice that unlike with the encoder, we don't need an `initialize_hidden_state()` method because the decoder's hidden state is initialized with the last hidden state of the encoder. Fill in the snippet following the instructions below.

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

class Decoder(nn.Module):

  def __init__(self, hidden_size, vocab_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(vocab_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, vocab_size)
    self.softmax = nn.LogSoftmax(dim=2)

  def forward(self, prev_word, hidden_state):
    embedding = self.embedding(prev_word).view(1,1,-1)
    output = F.relu(embedding)
    output, hidden = self.gru(output, hidden_state)
    output = self.softmax(self.out(output))
    return (output, hidden)

decoder = Decoder(HIDDEN_SIZE, len(tgt.vocab)).to('cuda')
print(decoder)

Decoder(
  (embedding): Embedding(2758, 128)
  (gru): GRU(128, 128)
  (out): Linear(in_features=128, out_features=2758, bias=True)
  (softmax): LogSoftmax(dim=2)
)


### The Constructor

As with the encoder, our decoder is a subclass of `nn.Module`. In addition to the the embedding layer and the GRU layer, we need to instantiate two other layers: an output layer and a (log) softmax "layer."

Fill in the constructor by instantiating an `nn.Embedding`, an `nn.GRU`, an `nn.Linear` (the output layer), and an `nn.LogSoftmax` (the (log) softmax "layer").

For the `nn.LogSoftmax` "layer", there is only one argument to the constructor: which dimension to compute the softmax over, ie. the values along that dimension will be squashed into the range $[0, 1]$ and all sum to 1. Make sure you fill in the correct dimension! If you put the wrong dimension, the code will still run without errors, but your model will not work correctly.

### The Forward Pass

Similarly to the encoder, the decoder's forward pass should run the input through the embedding layer, the GRU layer, the output layer, and the (log) softmax layer; the return value should be a (log) probability distribution over the vocabulary. Fill in the `forward()` method above. The key differences from the encoder's method are as follows:

- Because autoregressive generation occurs one word at a time, the decoder runs on a single word at a time, rather than a sequence of words. Thus the decoder takes the index of a single word, ie. the previous word from the output sequence, as input. As we discussed in class, this can either be the gold standard previous word $y_{i-1}$ or the model's own prediction $\hat{y}_{i-1}$.

- We can still ignore the `output` of the GRU layer; since we are running a single word at a time, `output` is exactly the same as the hidden state.

- We want to return both the output (log) probability distribution and the updated hidden state. The output distribution allows us to predict a word for this time step, while the updated hidden state will be passed to the decoder at the next time step.

# Training the Neural Network --- 15 points

Now that we've built the network, it's time to train! By convention, and to keep code organized, PyTorch training code is often broken into two functions, one that runs a single training example (or mini-batch) through the network, computes the loss, and performs an update, and a second that manages loading the training data and looping through the training set.

## Training on a Single Example

First we'll write the function to run a single training pair, consisting of one input sequence and one output sequence, through our encoder-decoder network.

You'll need to fill in the code snippet following the instructions below.

But first, let's talk about some of the arguments.

- `input_sequence` and `target_sequence`. These are `torch.Tensor`'s of shape `(sequence_length, 1)`, ie. they are matrices where each row represents a meaning representation or word and contains a single value, which is the index of that word in the corresponding source (input) or target vocabulary.

- `encoder_optimizer` and `decoder_optimizer`. These are instances of optimizer classes from the module `torch.optim`, which implement many of the adjustable learning rate algorithms we saw in class. For example, there are `torch.optim.SGD`, `torch.optim.Adagrad`, and `torch.optim.RMSProp`.

- `loss_function` is a class from the `torch.nn` module that implements a loss function. For example, there are `torch.nn.CrossEntropyLoss` and `torch.nn.CosineEmbeddingLoss`.

The first step in the `train()` function, resetting the computational graphs of `encoder_optimizer` and `decoder_optimizer` using `zero_grad()`, has already been completed. The last steps and return statement have also been completed. Your task is to fill in the sections before and after the statement `loss = 0` to finish the function.

In [26]:
import random
from collections import defaultdict

def train(input_sequence, target_sequence, target_length, tgt_vocab,
          encoder, decoder, encoder_optimizer, decoder_optimizer,
          loss_function, teacher_forcing_ratio=0.5):
  
  encoder_hidden = encoder.initialize_hidden_state()
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  loss = 0
  input_length = input_sequence[-1].size(0)
  for i in range(input_length):
    encoder_hidden = encoder(input_sequence[i], encoder_hidden)[0]

  decoder_hidden = encoder_hidden
  prev_word = torch.tensor([[tgt_vocab.stoi[START]]], device='cuda')
  
  if random.random() < teacher_forcing_ratio:
    teacher_forcing_flag = True 
  else:
    teacher_forcing_flag = False

  for i in range(target_length):
     decoder_output, decoder_hidden = decoder(prev_word, decoder_hidden)
     top_values, top_indices = decoder_output.topk(1) 
     prev_word = top_indices.squeeze().detach()
     loss += loss_function(decoder_output[0], target_sequence[0,i].unsqueeze(0))
     if teacher_forcing_flag:
       prev_word = target_sequence[0,i]
     if prev_word.item() == tgt_vocab.stoi[END]:  
       break    
        
  loss.backward()
  encoder_optimizer.step()
  decoder_optimizer.step()
  return loss.item() / target_length

### Running the Encoder and Setting Up the Decoder



You can fill in the middle of this function following these steps:

1. Initialize the encoder hidden state and run `input_sequence` through the encoder.

2. Set the decoder hidden state equal to the updated encoder hidden state.

3. Initialize the first decoder `prev_word` $y_0$ to be the start meta-token. Recall that a word is represented not by a one-hot vector, as in the previous assignment, but by its index in the vocabulary. Thus `prev_word` should be a `torch.Tensor` containing a single value, the index of the start meta-token in the target vocabulary. You can look up the index corresponding to a given word using the dictionary `tgt_vocab.stoi` (standing for "string to index"), which is provided by `torchtext`. Make sure you set the option `device='cuda'` as well so that PyTorch puts this tensor on the GPU.

**WARNING:** Be very careful with the shape of `prev_word`. Look back at the code for `Encoder` and `Decoder` if you're not sure what the shape should be. If the shape is not correct, you will get an error when you try to train.

### Running the Decoder Loop

Now everything is set up to run the decoder. Recall that we will run the decoder on one `prev_word` at a time. It will output a (log) probability distribution over the vocabulary, which we can use to generate the predicted word $\hat{y}_i$. So we want to implement a loop that does the following:

1. Run the `prev_word` $y_{i-1}$ through the decoder to get both the (log) probability distribution and the updated decoder hidden state $h_i$.

2. Compute the loss using `loss_function()`. A call to `loss_function()` takes two `torch.Tensor` arguments: the output of the decoder and the gold standard target word $y_i$ from `target_sequence`. Add this value to the variable `loss`.

3. Decide if you are using teacher forcing ($y_i$) or not ($\hat{y}_i$) for the next time step. The argument `teacher_forcing_ratio` tells you the probability that a given decoder step uses teacher forcing. You can use the `random` module to make this decision.

4. If using teacher forcing, the new `prev_word` should be the gold standard target word. If not, then the new `prev_word` is the predicted word. At training time, the predicted word is simply the word with the highest output probability (ie.  greedy decoding). You can get this word using `torch.topk()` (see [the documentation here](https://pytorch.org/docs/stable/generated/torch.topk.html)). You will also have to call `detach()` to unlink it from the computation graph that generated this output, otherwise the computation graph for the current time step $i$ will get connected to the new computation graph for the next time step $i+1$, which will blow up our training time because we now need to do backpropagation through longer and longer graphs the more words we predict. This detach step is part of the truncated backpropagation through time, which we discussed in class.

5. Repeat the loop with the new `prev_word` and updated decoder hidden state until you have predicted a word for every position in the target sequence.

After this loop runs, the last lines that have already been completed will execute: running the backward pass using `loss.backward()`, performing parameter updates using `optimizer.step()`, and returning the average word-level loss for this training pair.

**WARNING:** Far and away the most common source of error when implementing a neural network is incorrect tensor shape. Be very careful when creating a new tensor, eg. the first `prev_word`, or when slicing an existing tensor, eg. to get $y_i$ from `target_sequence`, that the shape of your tensor matches what the rest of the network expects. Carefully read the input and output tensor shapes of the PyTorch embedding and GRU layers. You can use the functions `torch.unsqueeze()` and `torch.squeeze()` to add or get rid of extra dimensions in your tensors (see the documentation [here](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html) and [here](https://pytorch.org/docs/stable/generated/torch.squeeze.html)). When in doubt, print `Tensor.size()` to see if the shape makes sense.

## Training an Epoch

The code snippet below is already complete; you don't need to fill in anything. It initializes our encoder and decoder optimizers (we are using vanilla gradient descent), as well as the loss function, negative log likelihood loss. It then loads the training data using a `torchtext.data.Iterator` object and runs each training pair through our network using the `train()` function we just wrote.

In [27]:
from torch import optim

def train_epochs(train_data, tgt_vocab, encoder, decoder,
                num_epochs=1,
                learning_rate=0.01, teacher_forcing_ratio=0.5):
  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
  loss_function = nn.NLLLoss()

  data_generator = torchtext.data.Iterator(
      dataset=train_data,
      batch_size=1,
      device='cuda',
  )

  for e in range(num_epochs):
    print('Epoch %d' %e)
    count, epoch_loss = 0, 0
    for training_example in data_generator.__iter__():
      input_sequence = getattr(training_example, 'src')
      target_sequence, target_length = getattr(training_example, 'tgt')

      epoch_loss += train(input_sequence, target_sequence, target_length, tgt_vocab,
                          encoder, decoder, encoder_optimizer, decoder_optimizer,
                          loss_function, teacher_forcing_ratio)
      count += 1
    print('Loss %f' % (epoch_loss.item() / count))

We are now ready to train the model using the code snippet below. We are running on the GPU this time, so training is much faster; one epoch should take around 15-25 minutes. If you want to run multiple epochs at a time, feel free to increase `num_epochs`.

In [28]:
import datetime

print(datetime.datetime.now())
train_epochs(train_data, tgt.vocab, encoder, decoder, num_epochs=1)
print(datetime.datetime.now())

2020-11-26 05:08:14.626321
Epoch 0
Loss 2.606466
2020-11-26 05:26:36.385352



**WARNING:** Error messages tend to be unhelpful when running on the GPU. This is partly because the GPU executes operations in parallel, and it can also execute asynchronously from the CPU. As a result, you often can't tell which line of code is actually causing the error; the stack trace can be out of sync with GPU execution. Another reason is that, once a tensor is loaded on the GPU, its dimension and/or indexing errors become GPU memory access errors, and memory errors of any kind are difficult to debug. 

If you are still in the process of debugging your code, it is much easier to do so on the CPU. You can run on the CPU by removing the `device='cuda'` options and `to('cuda')` calls. Once your code runs successfully on the CPU, you can switch back to the GPU.

If you do get a GPU memory error (ie. it is a runtime error with `cuda`, `cublas`, or `cudnn` in the name), you will need to reset your Colab runtime using the Runtime menu to clear the GPU memory before you can run again. Otherwise the GPU memory will remain in its errorful state, and you will constantly get weird error messages, even on code snippets that previously worked fine.

Once you've successfully trained, you can use the code snippets below for saving and loading the model so that you can save your progress and also continue training more epochs as time allows. Notice that we only save/load `state_dict`, which is the dictionary where PyTorch stores the current parameter values of a model. We can easily instantiate a new encoder/decoder and overwrite its randomly initialized parameters with our saved values, so saving only the parameters is more space-efficient than saving the entire network.

In [29]:
# Save and download the trained model parameters

torch.save(encoder.state_dict(), 'hw5.encoder')
torch.save(decoder.state_dict(), 'hw5.decoder')

from google.colab import files
files.download('hw5.encoder')
files.download('hw5.decoder')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
# Upload saved model parameters

from google.colab import files

files.upload()
encoder.load_state_dict(torch.load('/content/hw5.encoder'))
decoder.load_state_dict(torch.load('/content/hw5.decoder'))

Saving hw5.decoder to hw5 (1).decoder
Saving hw5.encoder to hw5 (1).encoder


<All keys matched successfully>

# Decoding --- 5 points

Now that we have a trained model, we can generate some restaurant descriptions! Fill in the function below to generate a resturant description from an input list of meaning representations (strings). As before, you want to run the input through the encoder, then set up the decoder and run the decoder loop, generating the highest (log) probability word at each iteration (ie. greedy decoding); you should be able to reuse some of your code from `train()`. The key differences from `train()` are as follows:

- You should wrap the entire contents of this function with a `with` statement that sets `torch.no_grad()`. This will tell PyTorch that it doesn't need to keep track of the computation graph values and gradients. Since we aren't doing parameter updates anymore, we don't need the gradients, so telling PyTorch to skip calculating them will make this function run faster.

- In the decoder loop, you should stop generating words and return if the end meta-token is generated before `max_length` is reached; otherwise, you should generate `max_length` words and then return. Don't include the start meta-token or the end meta-token, if generated, in the returned list of words.

- The return type of this function is a list of strings, but decoder predicts a tensor containing an index into the target vocabulary. You will need to first convert this tensor into an int using `Tensor.item()`, and then convert the int into a word using the `torchtext` dictionary `tgt_vocab.itos` ("index to string").

As in `train()`, be very careful with the shapes of your tensors. Most errors in neural network implementation are due to tensor shape mismatch.

In [35]:
# Fill in this function
# The argument input_sequence is a torch.Tensor
# The arguments src_vocab and tgt_vocab are torchtext.vocab.Vocab objects
# The arguments encoder and decoder are our Encoder and Decoder
# The argument max_length is an int
# The return type should be a list of strings
def generate(input_sequence, src_vocab, tgt_vocab, encoder, decoder, max_length=MAX_LEN):
    with torch.no_grad():
        input_length = input_sequence[1].size()[0]

        encoder_hidden = encoder.initialize_hidden_state()
        input_length = input_sequence[-1].size(0)

        for i in range(input_length):
          encoder_hidden = encoder(input_sequence[i].view(1,-1), encoder_hidden)[0]
        decoder_hidden = encoder_hidden
        prev_word = torch.tensor([[tgt_vocab.stoi[START]]], device='cuda')
        decoded_words = []
        
        for i in range(max_length):
          decoder_output, decoder_hidden = decoder(prev_word, decoder_hidden)
          topv, topi = decoder_output.data.topk(1)
          next_word = tgt_vocab.itos[topi.item()]
          decoded_words.append(next_word)
          if next_word == '</s>':
             break
          decoder_input = topi.squeeze().detach()
        return decoded_words[1:]

for i in range(5):
    input = random.choice(train_data.examples).src
    print(input)
    input_sequence = torch.tensor([src.vocab.stoi[word] for word in input], device='cuda').unsqueeze(1)
    print(' '.join(generate(input_sequence, src.vocab, tgt.vocab, encoder, decoder)))
    print()

['name[Strada]', ' food[Indian]', ' priceRange[£20-25]', ' customer rating[high]', ' familyFriendly[no]', ' near[Rainbow Vegetarian Café]']
strada strada strada strada the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

['name[The Waterman]', ' food[English]', ' familyFriendly[yes]', ' near[Crowne Plaza Hotel]']
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

['name[The Waterman]', ' food[Italian]', ' priceRange[cheap]', ' customer rating[average]', ' area[city centre]', ' familyFriendly[yes]']
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

['name[The Punter]', ' ea

You should get some fairly okay results on the training set after a single epoch, and the fluency of the generated descriptions will improve with subsequent epochs of training.

# Evaluation --- 5 points

The last thing we need to do is evaluate the performance of the model. We will use the BLEU metric, which is commonly used for translation-type tasks, like this one. The formula for BLEU is 

$\text{min} \left( 1, \cfrac{\text{generation length}}{\text{target length}} \right) \left( \displaystyle{\prod_{i=1}^4} \text{precision}_i \right)^{\frac{1}{4}}$

where $\text{precision}_i$ refers to the n-gram precision with $n=i$.

For these n-grams, we won't add any meta-tokens to pad the beginning and end of the sentences; we only count the words that actually occur in our generated sentence and the target sentence. 

Fill in the code snippet below to implement BLEU, following the formula above.

In [36]:
# Fill in this function
# The arguments generated_words and target_words are lists of strings
# The return type should be a float
import nltk
def bleu(generated_words, target_words, n=4):
  prefix = min(1,len(generated_words)/len(target_words))
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([target_words], generated_words)
  return BLEUscore


This last code snippet will calculate the average BLEU score of your model across the entire test set. You should get okay performance after just one epoch, and your score will increase with more training.

In [37]:
# Calculate average BLEU over the test set

num_test = len(test_data.examples)
total_bleu = 0.

for test_example in test_data.examples:
  input, target = test_example.src, test_example.tgt
  input_sequence = torch.tensor([src.vocab.stoi[word] for word in input], device='cuda').unsqueeze(1)
  total_bleu += bleu(generate(input_sequence, src.vocab, tgt.vocab, encoder, decoder), target)

print(total_bleu / num_test)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.40961671975673625


All done! Make sure your trained model parameters are saved, and use the "File" menu to download this notebook for GradeScope submission.